In [16]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Does nn.Conv2d init work well?

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=21)

In [17]:
#export
from exp.nb_02 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [18]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [19]:
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [20]:
x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)
x_train.shape,x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [21]:
n,*_ = x_train.shape
c = y_train.max()+1
nh = 32
n,c

(50000, tensor(10))

In [78]:
l1 = nn.Conv2d(1, nh, 5)

In [79]:
x = x_valid[:100]

In [80]:
x.shape

torch.Size([100, 1, 28, 28])

In [81]:
def stats(x): return x.mean(),x.std()

In [82]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [83]:
stats(l1.weight),stats(l1.bias)

((tensor(0.0020, grad_fn=<MeanBackward0>),
  tensor(0.1153, grad_fn=<StdBackward0>)),
 (tensor(0.0312, grad_fn=<MeanBackward0>),
  tensor(0.0944, grad_fn=<StdBackward0>)))

In [84]:
stats(x)

(tensor(-0.0363), tensor(0.9602))

In [85]:
t = l1(x)

In [86]:
stats(t)

(tensor(0.0360, grad_fn=<MeanBackward0>),
 tensor(0.5219, grad_fn=<StdBackward0>))

In [87]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(0.0232, grad_fn=<MeanBackward0>),
 tensor(1.1411, grad_fn=<StdBackward0>))

In [88]:
import torch.nn.functional as F

In [89]:
def f1(x,a=0): return F.leaky_relu(l1(x),a)

In [95]:
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.4836, grad_fn=<MeanBackward0>),
 tensor(0.9765, grad_fn=<StdBackward0>))

In [96]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2542, grad_fn=<MeanBackward0>),
 tensor(0.3945, grad_fn=<StdBackward0>))

In [97]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [98]:
# receptive field size
rec_fs = l1.weight[0,0].numel()
rec_fs

25

In [99]:
nf,ni,*_ = l1.weight.shape
nf,ni

(32, 1)

In [100]:
fan_in  = ni*rec_fs
fan_out = nf*rec_fs
fan_in,fan_out

(25, 800)

In [105]:
def gain(a): return math.sqrt(2.0 / (1 + a**2))

In [106]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

In [107]:
torch.zeros(10000).uniform_(-1,1).std()

tensor(0.5741)

In [108]:
1/math.sqrt(3.)

0.5773502691896258

In [112]:
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

In [113]:
kaiming2(l1.weight, a=0);
stats(f1(x))

(tensor(0.5630, grad_fn=<MeanBackward0>),
 tensor(1.0999, grad_fn=<StdBackward0>))

In [115]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.2413, grad_fn=<MeanBackward0>),
 tensor(0.3872, grad_fn=<StdBackward0>))

In [116]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

In [118]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [119]:
y = y_valid[:100].float()

In [122]:
t = m(x)
stats(t)

(tensor(-0.0070, grad_fn=<MeanBackward0>),
 tensor(0.0088, grad_fn=<StdBackward0>))

In [123]:
l = mse(t,y)
l.backward()

In [125]:
stats(m[0].weight.grad)

(tensor(-0.0011), tensor(0.0738))

In [126]:
init.kaiming_uniform_??

In [135]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [136]:
t = m(x)
stats(t)

(tensor(-0.1808, grad_fn=<MeanBackward0>),
 tensor(0.2864, grad_fn=<StdBackward0>))

In [137]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(0.0149), tensor(0.5199))

## Export

In [ ]:
!./notebook2script.py 02a_why_sqrt5.ipynb